# Document Classification with Naive Bayes - Lab

## Introduction

In this lecture, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

You will be able to:  

* Implement document classification using naive Bayes
* Understand the need for the Laplacian smoothing correction
* Explain how to code a bag of words representation

## Import the Dataset

To start, import the dataset stored in the text file `SMSSpamCollection`.

In [8]:
#Your code here

import pandas as pd
import numpy as np
data = pd.read_csv('SMSSpamCollection', sep = '\t', header=None)
data.columns = ['target','text']
data.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for Class Imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [64]:
p_class = dict(train_df.target.value_counts(normalize=True))

In [47]:
#Your code here
ham = data[data['target'] == 'ham']
spam = data[data['target'] =='spam']
ind = np.random.choice(ham.index, len(spam),replace = False)
ham_keep = ham.loc[ind,:]
data_filtered = pd.concat([ham_keep,spam])

## Train - Test Split

Now implement a train test split on your dataset.

In [48]:
from sklearn.model_selection import train_test_split
X= data_filtered.text
y = data_filtered.target

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
train_df = pd.concat([X_train, y_train], axis=1) 
test_df = pd.concat([X_test, y_test], axis=1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class.

In [53]:
#Your code here
word_frequency ={}
for t in train_df.target.unique():
    word_count = {}
    for i in range(len(train_df)):
        doc = train_df.iloc[i]['text']
        for word in doc.split():
            word_count[word]= word_count.get(word,0)+1
    word_frequency[t]=word_count

In [54]:
word_frequency

{'ham': {'I': 190,
  'think': 13,
  'that': 52,
  "tantrum's": 1,
  'finished': 2,
  'so': 54,
  'yeah': 4,
  "I'll": 16,
  'be': 72,
  'by': 41,
  'at': 53,
  'some': 20,
  'point': 2,
  'Oh': 9,
  'and': 165,
  'the': 264,
  'way': 14,
  'you': 321,
  'do': 25,
  'have': 154,
  'more': 17,
  'food': 3,
  'in': 123,
  'your': 201,
  'fridge!': 1,
  'Want': 9,
  'to': 643,
  'go': 39,
  'out': 54,
  'for': 194,
  'a': 385,
  'meal': 1,
  'tonight?': 4,
  'Hey': 13,
  'sathya': 1,
  'till': 7,
  'now': 68,
  'we': 34,
  'dint': 1,
  'meet': 7,
  'not': 52,
  'even': 10,
  'single': 2,
  'time': 30,
  'then': 27,
  'how': 14,
  'can': 54,
  'i': 87,
  'saw': 2,
  'situation': 3,
  'sathya.': 1,
  'Thanks': 17,
  'ringtone': 11,
  'order,': 4,
  'reference': 3,
  'number': 24,
  'X49.': 1,
  'Your': 55,
  'mobile': 60,
  'will': 62,
  'charged': 9,
  '4.50.': 3,
  'Should': 5,
  'tone': 13,
  'arrive': 5,
  'please': 18,
  'call': 168,
  'customer': 22,
  'services': 7,
  '09065989182.': 

## Count the Total Corpus Words
Calculate V, the total number of words in the corpus.

In [59]:
#Your code here
V = set()
for t in word_frequency.keys():
    for word in word_frequency[t].keys():
        V.add(word)
total = len(V)
total      

5953

## Create a Bag of Words Function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [60]:
#Your code here
def bag_it(doc):
    bag_words ={}
    for word in doc.split():
        bag_words[word]= bag_words.get(word,0)+1
    return bag_words
        

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [71]:
#Your code here
def classify_doc(doc, word_frequency, p_class, total, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in word_frequency.keys():
        p = np.log(p_class[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = word_frequency[class_].get(word, 0) + total
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test Out Your Classifier

Finally, test out your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [75]:
#Your Code here
prediction_test = classify_doc(test_df.iloc[1]['text'], word_frequency, p_class, total,return_posteriors=True)
prediction_test

[-143.2799449155299, -143.30494621773533]


'ham'

In [85]:
y_hat_test = X_test.map(lambda x: classify_doc(x, word_frequency, p_class, total))
residuals = y_test == y_hat_test
residuals.value_counts(normalize=True)


False    0.518717
True     0.481283
dtype: float64

## Level-Up

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing naive Bayes' for document classification!